# GFS Forecast Maps from Thredds Server via NCSS and Siphon
# MSLP w/ Highs and Lows and Thickness

## Justin Richling
## 11/07/19
* Included updates to THREDDS catalog heirarchy

https://doi.org/10.6084/m9.figshare.5244637.v1

In [19]:
# System Tools
import os

# Importing Datetime Libraries
from datetime import datetime, timedelta

# CartoPy Map Plotting Libraires
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Numerical and Scientific Libraries
import numpy as np
from scipy.ndimage import gaussian_filter

# Accessing Data from XLM Catalog via Siphon Libraries
from siphon.catalog import TDSCatalog
from siphon.ncss import NCSS

# MetPy Libraries
from metpy.plots import add_metpy_logo

# NetCDF Libraries
from netCDF4 import num2date

# Matplotlib Plotting Libraries
import matplotlib.pyplot as plt
from matplotlib import patheffects

# Warnings
import warnings
warnings.filterwarnings('ignore')

In [20]:
# Set the font 
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 18,
        }

## Helper Functions

In [21]:
# Thanks to the crew over at Metpy for this handy little function
def find_time_var(var, time_basename='time'):
    for coord_name in var.coordinates.split():
        if coord_name.startswith(time_basename):
            return coord_name
    raise ValueError('No time variable found for ' + var.name)

In [22]:
cd ~/Downloads/

/Users/chowdahead/Downloads


In [23]:
from HiLo import plot_maxmin_points as HL

<h2>----------------------------------------------//---------------------------------------------------------</h2>

## Set the Map Projection

In [24]:
# Set Projection of Data
datacrs = ccrs.PlateCarree()

# Set Projection of Plot
plotcrs = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)

# Add Map Features
states_provinces = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_1_states_provinces_lakes',scale='50m', facecolor='none')

country_borders = cfeature.NaturalEarthFeature(category='cultural',
    name='admin_0_countries',scale='50m', facecolor='none')

# Colorbar Axis Placement (under figure)
colorbar_axis = [0.183, 0.09, 0.659, 0.03] # [left, bottom, width, height]

# Lat/Lon Extents [lon0,lon1,lat0,lat1]
extent = [-130., -70, 20., 60.]

<h2>----------------------------------------------//---------------------------------------------------------</h2>

## Figure out where the saved maps will go

In [25]:
now = datetime.utcnow()
#now = datetime(2019,4,10,0,0)
today_day = int('{0:%d}'.format(now))
today_year = int('{0:%Y}'.format(now))
today_month = int('{0:%m}'.format(now))
print(today_day,today_year,today_month)


# Set a path to save the plots with string format for the date to set the month and day
im_save_path ="/path/to/saved/images/"
im_save_path =f"/Users/chowdahead/Desktop/Weather_Blog/{today_year}/{today_month}_{today_day}/"
print(im_save_path)

# Check to see if the folder already exists, if not create it
if not os.path.isdir(im_save_path):
    os.makedirs(im_save_path)

# Uncomment if you want to automatically change to the map folder    
#os.chdir(im_save_path)

19 2019 11
/Users/chowdahead/Desktop/Weather_Blog/2019/11_19/


<h2>----------------------------------------------//---------------------------------------------------------</h2>

<h1><font style="font-size:32px"><center>-- Plotting all of the GFS forecast hours for the current day --</center></font></h1>

<h2><font><center>-- Highs/Lows and 1000-500mb Thickness --</center></font></h2>

<h2>----------------------------------------------//---------------------------------------------------------</h2>

In [26]:
# Request the GFS data from the thredds server
gfs = TDSCatalog('http://thredds-jetstream.unidata.ucar.edu/thredds/catalog/grib/'
                 'NCEP/GFS/Global_0p5deg/catalog.xml')

print(list(gfs.datasets.values()))
dataset = list(gfs.datasets.values())[1]
#print(dataset.access_urls)
print(dataset)

[Full Collection (Reference / Forecast Time) Dataset, Best GFS Half Degree Forecast Time Series, Latest Collection for GFS Half Degree Forecast]
Best GFS Half Degree Forecast Time Series


In [31]:
# Create NCSS object to access the NetcdfSubset
ncss = NCSS(dataset.access_urls['NetcdfSubset'])
# get current date and time
#now = forecast_times[0]
now = datetime(today_year,today_month,today_day,0,0)
# define time range you want the data for
start = now
print(start)
delt_t = 48
end = now + timedelta(hours=delt_t)

query = ncss.query()
query.time_range(start, end)
query.lonlat_box(north=60, south=20, east=310, west=230)
query.accept('netcdf4')
query.variables("MSLP_Eta_model_reduction_msl","Geopotential_height_isobaric")


# Request data for the variables you want to use
data = ncss.get_data(query)

# Pull out the lat and lon data
lats = data.variables['lat'][:]
lons = data.variables['lon'][:]

# Get time into a datetime object
time_var = data.variables[find_time_var(data.variables['MSLP_Eta_model_reduction_msl'])]
time_var = num2date(time_var[:], time_var.units).tolist()
time_strings = [t.strftime('%m/%d %H:%M') for t in time_var]

# Combine 1D latitude and longitudes into a 2D grid of locations
lon_2d, lat_2d = np.meshgrid(lons, lats)

time_var = data.variables[find_time_var(data.variables["MSLP_Eta_model_reduction_msl"])]
time_final = num2date(time_var[:].squeeze(), time_var.units)

mslp_name = "MSLP_Eta_model_reduction_msl"
hgt_name = "Geopotential_height_isobaric"

mslp = data.variables[mslp_name][:]/100
mslp = gaussian_filter(mslp, sigma=3.0)
hgt = data.variables[hgt_name][:]

print("done.")

2019-11-19 00:00:00
done.


In [38]:
def HiLo_Thck(time_index):

                                # Setup Contour Label Options
#---------------------------------------------------------------------------------------------------    
    kw_clabels = {'fontsize': 11, 'inline': True, 'inline_spacing': 5, 'fmt': '%i',
              'rightside_up': True, 'use_clabeltext': True}

                                    # Setup Figure
#---------------------------------------------------------------------------------------------------    
    fig = plt.figure(figsize=(17., 11.))

    add_metpy_logo(fig, 25, 925, size='small')

                                    # Add the Map 
#---------------------------------------------------------------------------------------------------
    ax = plt.subplot(111, projection=plotcrs)

# Set extent and plot map lines
    ax.set_extent(extent, datacrs)
    
    ax.coastlines(resolution='50m')

                            # Add State/Country Boundaries to Plot
#---------------------------------------------------------------------------------------------------    
    state_borders = cfeature.NaturalEarthFeature(
                category='cultural', name='admin_1_states_provinces_lines',
                scale='50m', facecolor='none')
    ax.add_feature(state_borders, edgecolor='b', linewidth=1, zorder=3)
    
    country_borders = cfeature.NaturalEarthFeature(category='cultural',
                name='admin_0_countries',scale='50m', facecolor='none')
    ax.add_feature(country_borders,edgecolor='b',linewidth=0.2)
    
    ax.add_feature(cfeature.STATES)
    ax.add_feature(cfeature.BORDERS)
        
                                    # File and Title Times
#---------------------------------------------------------------------------------------------------
    # Time index for data variables
    time = time_strings[time_index]
    
    # Set string for saved image file name
    file_time = str(time_final[0]).replace("-","_").replace(" ","_").replace(":","")[:-2]+"Z"
    
    # Set forecast date and hour  
    forecast_date = "{}".format(today_year)+'-'+time_strings[time_index].replace("/","-")[:-5]
    forecast_hour = time_strings[time_index][-5:]+"Z"
    
    # Set initialization date and hour 
    init_date = "{}".format(today_year)+'-'+time_strings[0].replace("/","-")[:-5]
    init_hour = time_strings[0].replace("/","-")[-5:]+"Z"
    
    
                                        # Plot Title
#---------------------------------------------------------------------------------------------------
    ax.set_title('GFS 0.5$^{o}$: MSLP w/ Highs/Lows and\n1000-500mb Thickness (m)', 
                 size=16, loc='left',fontdict=font)
    
    ax.set_title(f"Init Hour: {init_date} {init_hour}\nForecast Hour: {forecast_date} {forecast_hour}",
                 size=16, loc='right',fontdict=font)

    
                                    # 1000-500mb Thickness
#---------------------------------------------------------------------------------------------------
    hgt_500 = hgt[time_strings.index(time),data.variables["isobaric6"][:].tolist().index(50000),:,:]
    hgt_1000 = hgt[time_strings.index(time),data.variables["isobaric6"][:].tolist().index(100000),:,:]
    thickness_1000_500 = gaussian_filter(hgt_500 - hgt_1000, sigma=3.0)
    
    clevs = (np.arange(0, 5400, 60), np.arange(5400,5401,1), np.arange(5460, 7000, 60))
    colors = ('tab:blue', 'b', 'tab:red')
    
    for clevthick, color in zip(clevs, colors):
        cs = ax.contour(lons, lats, thickness_1000_500, levels=clevthick, colors=color,
                    linewidths=1.0, linestyles='dashed', transform=datacrs)
        plt.clabel(cs, **kw_clabels)

                                           # MSLP
#---------------------------------------------------------------------------------------------------
    clevmslp = np.arange(800., 1120., 4)
    cs2 = ax.contour(lons, lats, mslp[time_strings.index(time),:,:], clevmslp, colors='k', linewidths=1.25,
                 linestyles='solid', transform=datacrs)
    plt.clabel(cs2, **kw_clabels)

                                    # High and Low Symbols
#---------------------------------------------------------------------------------------------------
    HL(ax,lon_2d, lat_2d, mslp[time_strings.index(time),:,:], 'max', 50, symbol='H',color='b',  transform=datacrs)
    HL(ax,lon_2d, lat_2d, mslp[time_strings.index(time),:,:], 'min', 25, symbol='L',color='r', transform=datacrs)
    
    
                                        # Save Figure
#---------------------------------------------------------------------------------------------------    
    GFS_HILO = im_save_path+"GFS/HILO/"
    if not os.path.isdir(GFS_HILO):
        os.makedirs(GFS_HILO)
        
    time_index *= 3
    if time_index < 10:
        times = f"0{time_index}"
    else:
        times = f"{time_index}"

    outfile = f"{GFS_HILO}GFS_0p5_HL_Thickness_{file_time}_F{times}.png"
    fig.savefig(outfile,bbox_inches='tight',dpi=120)
    plt.close(fig)
    
    
    

In [39]:
for i in range(0,int(delt_t/3)):
    HiLo_Thck(i)
    print("next.")
print("All done.")

next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
next.
All done.
